In [ ]:
def regression(data, location): # location needs to be of the format 'C:/Users/jihun/Documents/''
    # This function uses the following models:
    
        # linear regression
        # 1. OLS
        # 2. lasso
        # 3. ridge
        # 4. elastic net
        # 5. least angle regression
        # 6. bayesian regression
        # 7. robust regression
        
        # kernel
        # 9. kernel ridge regression
        
        # nearest neighbors
        # 8. knn
        
        # support veector machine
        # 7. linear
        # 8. polynomial
        # 9. radial
        
        # gaussian process
        # 10 gaussian process regression
        
        # tree methods
        # 11. random forest
        # 12. adaboost
        # 13. gradient boosting
        
        # neural network
        # 15. multilayer perceptron
 
    # import necessary libraries
    import pickle
    import numpy as np
    import pandas as pd
    from sklearn.svm import SVR
    import matplotlib.pyplot as plt
    from sklearn.externals import joblib
    from sklearn.kernel_ridge import KernelRidge
    from sklearn.neural_network import MLPRegressor
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,  RBF
    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, Lars, BayesianRidge, HuberRegressor

    # define response and feature
    y = data.iloc[:,0] # response variable needs to be the first column
    X = data.iloc[:,1:]
    
    # split train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # model initializers
    models = [
        LinearRegression(),
        Ridge(),
        Lasso(),
        ElasticNet(),
        Lars(),
        BayesianRidge(),
        HuberRegressor(),
        KernelRidge(),
        KNeighborsRegressor(),
        GaussianProcessRegressor(),
        RandomForestRegressor(), 
        AdaBoostRegressor(), 
        GradientBoostingRegressor(),
        MLPRegressor()]
    
    # fit a standard scaler
    scaler = StandardScaler()
    scaler.fit_transform(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # parameter space for grid search
    parameter_grid=[
        # Linear Regression
        {'fit_intercept':[True]},
        # Lasso Regression
        {'alpha':np.linspace(0.1,1,10)}, # Constant that multiplies the L1 term. 
        # Ridge Regression
        {'alpha':np.linspace(0.1,1,10)}, # Regularization strength; Regularization improves the conditioning of the problem and reduces the variance of the estimates. Larger values specify stronger regularization
        # Elastic Net
        {'alpha':np.linspace(0.1,1,10),  
        'l1_ratio': np.linspace(0.1,0.9,9)},
        # Least angle regression
        {'fit_intercept':[True]}, # Regularizes the covariance estimate as (1-reg_param)*Sigma + reg_param*np.eye(n_features)
        # Bayesian Ridge
        {'alpha_1':[1e-6,1], # Hyper-parameter : shape parameter for the Gamma distribution prior over the alpha parameter
         'alpha_2':[1e-6,1], # Hyper-parameter : inverse scale parameter (rate parameter) for the Gamma distribution prior over the alpha parameter
         'lambda_1':[1e-6,1], # Hyper-parameter : shape parameter for the Gamma distribution prior over the lambda parameter
         'lambda_2':[1e-6,1]}, # Hyper-parameter : inverse scale parameter (rate parameter) for the Gamma distribution prior over the lambda parameter
        # Huber Regression
        {'alpha':[0.001,0.01,0.05,0.1,0.5,1]}, # Regularization parameter.
        # Kernel Ridge Regression
        {'gamma':['RBF', 'laplacian', 'polynomial', 'exponential chi2'], # Kernel mapping used internally
        'alpha':[0.001,0.01,0.05,0.1,0.5,1]}, # Small positive values of alpha improve the conditioning of the problem and reduce the variance of the estimates
        # K Nearest Neighbor
        {'n_neighbors': [3,5,7,9,11], # Number of neighbors to use
        'weights':['uniform','distance']}, # weight function used in prediction
        # Gaussian Process
        {'optimizer':['fmin_l_bbfgs_b']},
        # Support Vector Classifier with Linear Kernel
        {'C':[0.001,0.01,0.05,0.1,0.2,0.5,1]}, # regularization parameter; The strength of the regularization is inversely proportional to C. Must be strictly positive.
        # Support Vector Classifier with Polynomial Kernel
        {'C':[0.001,0.01,0.05,0.1,0.2,0.5,1],
        'degree':[1,2,3], # Degree of the polynomial kernel function
        'gamma':['scale','auto']}, # Kernel coefficient
        # Support Vector Classifier with RBF Kernel
        {'C':[0.001,0.01,0.05,0.1,0.2,0.5,1],
        'gamma':['scale','auto']}, 
        # Random Forest
        {'n_estimators': [500, 1000, 1500],
         'max_features': ['auto','log2'],
         'criterion' :['gini', 'entropy']},
        # Adaboost
        {'n_estimators': [500, 1000, 1500],
         'learning_rate':[0.01,0.05,0.1,0.5,1,2]}, # Learning rate shrinks the contribution of each regressor 
        # Gradient Boosting
        {'n_estimators': [500, 1000, 1500],
         'learning_rate':[0.01,0.05,0.1,0.5,1,2], # learning rate shrinks the contribution of each tree
         'loss':['deviance','exponential'],
         'subsample':[0.7,0.85,1.0], # The fraction of samples to be used for fitting the individual base learners
         'max_depth': [2,3,4,5], # maximum depth of the individual regression estimators.
         'max_features':['auto']},
        # Multilayer Perceptron
        {'solver': ['lbfgs'],
        'learning_rate': ["constant", "invscaling", "adaptive"], # Learning rate schedule for weight updates
        'hidden_layer_sizes':[(5,5,5), (7,5,3), (10,)], # The ith element represents the number of neurons in the ith hidden layer
        'alpha': [0.001,0.01,0.1,1], # L2 penalty (regularization term) parameter
        'activation': ["relu", "tanh", 'identity'], # Activation function for the hidden layer
        'random_state':[97]}
    ] 

    modList = []
    result = np.repeat(1,3)
    
    colnames = ['r-squared',
               'mean squared error',
               'mean absolute error']
        
    rownames = [1,
                'linear regression',
                'lasso regression',
                'ridge regression',
                'elastic net',
                'least angle regression',
                'bayesian regression',
                'robust regression',
                'kernel ridge regression',
                'knn',
                'svm-linear',
                'svm-polynomial',
                'svm-rbf',
                'gaussian process',
                'random forest',
                'adaboost',
                'gradient boosting',
                'multilayer perceptron']
    
    # fit models via loop
    for clf,grid,name in zip(models, parameter_grid, rownames):
        # gridsearch
        mod = GridSearchCV(estimator=clf,
              param_grid=grid,
              cv=10)
        mod.fit(X_train,y_train)
        # save the model
        joblib.dump(clf, location+name+'.pkl') 
        modList.append(mod)
        # make prediction on test set
        predictions = mod.predict(X_test)  
        # calculate classification metrics
        metrics = np.array([
            accuracy_score(y_test, predictions),
            roc_auc_score(y_test, predictions),
            balanced_accuracy_score(y_test, predictions),
            cohen_kappa_score(y_test, predictions),
            f1_score(y_test, predictions),
            recall_score(y_test, predictions),
            precision_score(y_test, predictions),
            average_precision_score(y_test, predictions),
            brier_score_loss(y_test, predictions)
        ])
        # add the metrics to the result
        result = np.vstack((result,metrics))
        
    
    # save the final classification metric table

    result = pd.DataFrame(result, columns=colnames, index=rownames)
    result.to_csv(location + 'classification_model_results.csv')
    
    return result

        
